# Multimodal Prompt-to-Image Generation

이미지 생성 모델을 사용하여 원하는 이미지를 생성하려면 프롬프트에서 이미지의 세부 사항을 정확하게 설명하는 것이 중요합니다. 그러나 사전 정보 없이 자세한 프롬프트를 만드는 것은 어려울 수 있습니다. 이를 돕기 위해 Multimodal LLM을 통해 사용자가 제공하는 스타일과 키워드를 기반으로 한 창의적인 이미지 프롬프트 아이디어를 제안합니다. 이를 통해 사용자는 보다 직관적으로 이미지 프롬프트를 생성할 수 있으며, 프롬프트 작성에 소요되는 노력을 줄일 수 있습니다.

이미지 생성 모델은 [Amazon Titan Image Generator G1 v2](https://aws.amazon.com/ko/blogs/korea/amazon-titan-image-generator-v2-is-now-available-in-amazon-bedrock/)를 사용합니다.

In [ ]:
!pip install --quiet boto3 langchain

In [ ]:
import os
import sys

ROOT_PATH = os.path.abspath("../")
sys.path.append(ROOT_PATH)

In [ ]:
import re
import boto3

from common.aws.claude import BedrockClaude
from common.aws.titan_image import BedrockTitanImage, ImageParams
from common.utils.images import display_image, encode_image_base64_from_file

from langchain.prompts import PromptTemplate
from typing import List


boto3_session = boto3.session.Session(region_name='us-west-2')
print(f'The notebook will use aws services hosted in {boto3_session.region_name} region')

image_params = ImageParams()
titanImageGen = BedrockTitanImage(
    region=boto3_session.region_name,
    modelId='amazon.titan-image-generator-v2:0'
)

## Basic Prompt

기본 이미지 생성 프롬프트를 사용했을 때 입니다. Titan Image Generation 모델은 Input Text를 영어만 지원합니다.

In [ ]:
keyword="halloween",
style="realistic, fantastic"

body = image_params.text_to_image(text=f"{keyword}, {style}")
img = titanImageGen.generate_image(body)
display_image(img)

## LLM Prompt

LLM으로부터 Image Generation을 위한 프롬프트를 생성한 결과입니다. `style`, `keyword` 값을 입력하면, 해당 내용을 바탕으로 3개의 이미지 프롬프트를 제안합니다.

- `style`: 원하는 무드나 스타일
- `keyword`: 생성하고자 하는 이미지의 키워드

In [ ]:
PROMPT = """You are an Assistant that creates prompts for generate background image by image generator model. The image that Human wants is written in <keyword>.
Follow style guide in <style> and write three prompts for creating image keeping it to 300 characters or less. Use this fomat without further explanation: 
<prompt>image prompt</prompt>

<keyword>
{keyword}
</keyword>

<style>
{style}
</style>
"""

def extract_format(result_string):
    pattern = r'<prompt>(.*?)</prompt>'
    return re.findall(pattern, result_string)

def get_prompt(keyword: str, style: str): 
    return PromptTemplate(
                template=PROMPT,
                input_variables=["keyword", "style"]
            ).format(keyword=keyword,
                     style=style)

def get_prompt_by_llm(prompt: str) -> List[str]:
    claude = BedrockClaude()
    res = claude.invoke_llm_response(prompt)
    return extract_format(res)

In [ ]:
prompt = get_prompt(
    keyword=keyword,
    style=style
)

image_prompts = get_prompt_by_llm(prompt)
print(image_prompts)

In [ ]:
for prompt in image_prompts:
    print(f"[PROMPT] {prompt}")
    
    body = image_params.text_to_image(text=prompt)
    img = titanImageGen.generate_image(body)
    display_image(img)

## Multimodal LLM Prompt

Titan Image Generator 는 레퍼런스 이미지를 참조하여 새로운 이미지를 생성하는 몇 가지 옵션을 제공합니다.

- **Image Conditioning**: 텍스트 프롬프트를 따르면서 참조 이미지의 레이아웃 및 구성과 일치하는 이미지를 생성
  - 단순히 기존 이미지를 재구성하는 데 그칠 수 있어, 창의적인 자유도가 제한
- **Color Guided Generation**: `referenceImage`를 통해 참조 색상들을 받아와 이미지의 색상 톤을 반영
  - 일부 색상 톤을 반영하지만 이미지의 분위기나 재질 등을 반영하기 어려움
- **Image Variation**: 소스 이미지는 보존하고, 스타일과 배경을 변형한 이미지 생성
  - 변형의 범위가 제한적이고 원본 이미지에 제약이 있음

위 방법들은 레퍼런스 이미지의 특정 속성(색상, 레이아웃, 소스 이미지 등)을 반영하는 데에는 유용하지만, 변형의 범위가 제한적이다 보니 창의적이고 복합적인 특징을 반영한, 완전히 새로운 이미지를 생성하는 데에 한계가 있습니다. Multimodal LLM을 통해 레퍼런스 이미지를 이해하고, 이를 바탕으로 타겟하는 이미지를 생성하도록 프롬프트를 생성하면 더 자유롭고 창의적인 이미지 프롬프트 생성이 가능합니다.

In [ ]:
PROMPT = """You are an Assistant that creates prompts for generate background image by image generator model. The image that Human wants is written in <keyword>.
            Write a images three creation prompts keeping it to 300 characters or less to maintain the style of a given image. Use this fomat without further explanation:
            <prompt>image prompt</prompt>

            <keyword>
            {keyword}
            </keyword>
            """


def get_mm_prompt(keyword: str):
    return PromptTemplate(
                template=PROMPT,
                input_variables=["keyword"]
            ).format(keyword=keyword)

def get_prompt_by_mm_llm(prompt: str, image: str) -> List[str]:
    claude = BedrockClaude()
    res = claude.invoke_llm_response(text=prompt, image=image)
    return extract_format(res)

In [ ]:
input_image = encode_image_base64_from_file(f"./sample/house.jpg", format="JPEG")
display_image(input_image)

prompt = get_mm_prompt(keyword=keyword)
image_prompts = get_prompt_by_mm_llm(prompt=prompt, image=input_image)
print(image_prompts)

for prompt in image_prompts:
    print(prompt)
    body = image_params.text_to_image(text=prompt)
    img = titanImageGen.generate_image(body)
    display_image(img)